<a href="https://colab.research.google.com/github/amanchauhan786/RVFpga-TinyML-Work-IIT_ROORKEE-/blob/main/TFlite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install (if needed) and import required libraries
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

print("TensorFlow version:", tf.__version__)

# For Google Colab: Upload your CSV file
from google.colab import files
uploaded = files.upload()  # Choose your "data.csv" file here


TensorFlow version: 2.18.0


TypeError: 'NoneType' object is not subscriptable

In [ ]:
# Load your data
df = pd.read_csv("data.csv")

# Select features and labels
X = df[['temperature', 'humidity']].values
y = df['label'].values

# Scale features to [0, 1]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split: 15% test, 15% validation, 70% train
X_temp, X_test, y_temp, y_test = train_test_split(
    X_scaled, y, test_size=0.15, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.1765, random_state=42
)  # 0.1765 * 0.85 ≈ 0.15


In [ ]:
# Build a simple neural network
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(2,)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=30, batch_size=8,
    validation_data=(X_val, y_val),
    verbose=2
)


Epoch 1/30
44/44 - 5s - 109ms/step - accuracy: 0.4814 - loss: 0.6995 - val_accuracy: 0.8553 - val_loss: 0.6672
Epoch 2/30
44/44 - 1s - 11ms/step - accuracy: 0.8367 - loss: 0.6500 - val_accuracy: 0.8421 - val_loss: 0.6310
Epoch 3/30
44/44 - 1s - 13ms/step - accuracy: 0.8166 - loss: 0.6188 - val_accuracy: 0.8421 - val_loss: 0.5969
Epoch 4/30
44/44 - 0s - 10ms/step - accuracy: 0.8138 - loss: 0.5838 - val_accuracy: 0.8421 - val_loss: 0.5559
Epoch 5/30
44/44 - 0s - 7ms/step - accuracy: 0.8109 - loss: 0.5457 - val_accuracy: 0.8421 - val_loss: 0.5163
Epoch 6/30
44/44 - 1s - 15ms/step - accuracy: 0.8080 - loss: 0.5150 - val_accuracy: 0.8289 - val_loss: 0.4853
Epoch 7/30
44/44 - 1s - 19ms/step - accuracy: 0.8052 - loss: 0.4922 - val_accuracy: 0.8289 - val_loss: 0.4639
Epoch 8/30
44/44 - 0s - 9ms/step - accuracy: 0.8080 - loss: 0.4765 - val_accuracy: 0.8289 - val_loss: 0.4493
Epoch 9/30
44/44 - 1s - 17ms/step - accuracy: 0.8080 - loss: 0.4660 - val_accuracy: 0.8421 - val_loss: 0.4408
Epoch 10/30

In [ ]:
# Evaluate on validation and test sets
val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
print(f"\n✅ Validation Accuracy: {val_acc * 100:.2f}%")
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"✅ Test Accuracy: {test_acc * 100:.2f}%")



✅ Validation Accuracy: 89.47%
✅ Test Accuracy: 88.00%


In [ ]:
# Save the trained Keras model
model.save("base_model.h5")


In [ ]:
# FLOAT TFLite model
float_tflite_path = "anomaly_model.tflite"
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open(float_tflite_path, "wb") as f:
    f.write(tflite_model)
print(f"✅ Saved float TFLite model as: {float_tflite_path}")

# INT8 Quantized TFLite model
quant_tflite_path = "quant.tflite"
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
def rep_data_gen():
    for sample in X_train.astype(np.float32):
        yield [sample.reshape(1, 2)]
converter.representative_dataset = rep_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
quant_model = converter.convert()
with open(quant_tflite_path, "wb") as f:
    f.write(quant_model)
print(f"✅ Saved quantized TFLite model as: {quant_tflite_path}")


Saved artifact at '/tmp/tmprnu8gp5u'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 2), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  136423988629328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136423988632784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136423988634512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136423988632208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136423988634704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136423988630096: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ Saved float TFLite model as: anomaly_model.tflite
Saved artifact at '/tmp/tmpn5pl2xhq'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 2), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, n

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


✅ Saved quantized TFLite model as: quant.tflite


In [ ]:
assert os.path.exists(float_tflite_path), "❌ anomaly_model.tflite was not created!"
assert os.path.exists(quant_tflite_path), "❌ quant.tflite was not created!"
print("✅ Both TFLite models exist.")


✅ Both TFLite models exist.


In [ ]:
# Only for Colab users
try:
    from google.colab import files
    files.download(float_tflite_path)
    files.download(quant_tflite_path)
except Exception as e:
    print("Download not available (not in Colab or file not found).")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Test float TFLite model on test data
interpreter = tf.lite.Interpreter(model_path=float_tflite_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

y_pred = []
for sample in X_test:
    inp = np.array([sample], dtype=np.float32)
    interpreter.set_tensor(input_details[0]['index'], inp)
    interpreter.invoke()
    out = interpreter.get_tensor(output_details[0]['index'])[0][0]
    y_pred.append(1 if out > 0.5 else 0)

acc = accuracy_score(y_test, y_pred)
print(f"✅ TFLite Model Test Accuracy: {acc * 100:.2f}%")


✅ TFLite Model Test Accuracy: 88.00%
